# **Text Summarization testing**

To generate the notes from the extracted text, we thought of using some kind of abstractive text summarization which could generate a summary of the conversation keeping the relevant parts and feel human generated.

After applying a few different summarization algorithms we found the pretrained T5 - Text-To-Text Transfer Transformer to be satisfying our cause.

**Reason why we didn't use this**

The method generated a human like summary of the conversation but also kept missing various important part of the conversation which was not beneficial.

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 5.2 MB/s 
     |████████████████████████████████| 895 kB 42.8 MB/s 
     |████████████████████████████████| 3.3 MB 26.1 MB/s 
     |████████████████████████████████| 61 kB 381 kB/s 
     |████████████████████████████████| 596 kB 64.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 5.1 MB/s 


# Step 1

* Load pretrained model T5, we used 't5-small'

In [ ]:
import torch
import json 
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')
device = torch.device('cpu')

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

# Step 2


*   Generate text to summarise
*   Here we have created 2 representation of textual data
    
    (1) List of List
    
    (2) Paragraph 





In [ ]:
text = [['Yes, tell me. What problems you have?'],
['I am suffering from stomachache and motions since last night. I have also vomited a few times last night.'],
['What did you have yesterday?'],
['I had some snacks on the roadside stalls. It could be because of it.'],
['It is possible that you had contaminated food. Because of diarrhea, you have lost plenty of body fluids. You require to be hydrated. Drink enough water regularly, at least 10-12 glasses. Mix some Glucon-D powder or Electoral in water and have it. Fruit juice is also fine. Avoid caffeine, dairy products, and solid foods at least till evening. And get plenty of rest.'],
['Any medicines, doctor.'],
['Yes, I am prescribing a few to control diarrhea.'],
['Thank you, doctor.']]

'''text="""Doctor: So, what makes you come here?
Attendant: Doctor, my father has been diagnosed with unusual growth of cells in the colon, which other doctors so far have confirmed as a state of cancer.
Doctor: What tests have you conducted?
Attendant: On the recommendation of doctors in Delhi, we did blood tests, CT scan, and biopsy. We have taken three views on the reports and all have come up with opined colorectal cancer. Here are the reports.
Doctor: Yes, this seems cancerous. We need to admit him today itself and get a few tests done. And let’s meet tomorrow morning when you have the reports of these tests.
Attendant: OK. Thank you.
(The doctor formulates his observations on his letterhead and gives it over to the Patient. The person then admits the patient and visits the doctor again the next morning with new reports in hand.)
Doctor: The cancer is just one stage before it will expand to other parts. In this case, the best course of action is quick surgery to extract the affected part accompanied by radiation therapy.
Attendant: How many days of treatment will be needed?
Doctor: We can perform the operation in a couple of days. Consequently, we will keep the patient under observation for 4-5 days. Radiation therapy has become pretty regulated and therefore you can get it done in any tier-2 city, which will not only be useful to you but also decrease your cost.
Attendant: And for how long the patient will have to go through radiation therapy?
Doctor: The patient has to take one dose every two weeks for three months. You require to visit the hospital only on days you obtain therapy. Once radiation therapy is done, you should discuss an oncologist every three months in the beginning and every year later on to check for the remission of cancerous cells.
Attendant: OK. I’ve heard radiation therapy has side effects.
Doctor: Yes, radiation therapy has side effects. It can lead to hair loss, nausea, and loss of appetite.
Attendant: Thank you, doctor. We will get him admitted today."""'''


# Step 3


*   We used Paragraph form of textual data to create summary (natural way), but this represantation also kept missing various important part of the conversation which was not beneficial.
*   Then we used List of List to create sentence cluster so that summarisation is used repeate



In [ ]:
#preprocess_text = text.strip().replace("\n","")
Sentence_cluster=[]
i=0
while(i<len(text)):
  sent=""
  #print(text[i])
  for j in range(i,min(len(text),i+2)):
    sent+=text[j][0].strip().replace("\n","")
  i+=2  
  Sentence_cluster.append(sent)

In [ ]:
#print(Sentence_cluster)
for i in Sentence_cluster:
  t5_prepared_Text = "summarize: "+i
  tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)


  # summmarize 
  summary_ids = model.generate(tokenized_text,
                                  num_beams=4,
                                  no_repeat_ngram_size=2,
                                  min_length=30,
                                  max_length=100,
                                  early_stopping=True)

  output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

  print("Summary: ",output)

Summary:  i am suffering from stomachache and motions since last night. I have also vomited a few times last evening - and am also suffering stumblings and vomiting.
Summary:  i had some snacks on the roadside stalls. it could be because of it. I had a few snacks yesterday.
Summary:  it is possible that you had contaminated food because of diarrhea. you need to be hydrated, drink enough water regularly, at least 10-12 glasses.
Summary:  i am prescribing a few to control diarrhea.thank you, doctor, for the advice. if you have any questions, please do not hesitate to contact me.
